In [235]:
load('Imports/Utility.sage', 'Imports/Point.sage', 'Imports/Line.sage', 'Imports/Tritangent_plane.sage', 'Imports/Group.sage', 'Imports/Cubic.sage')
import multiprocessing as mp
import pickle

In [51]:
P.<x,y,z,t,b,c,d,e,f,l,m> = PolynomialRing(QQ)

In [52]:
cubic_new = e*f*(2*x^2*y-2*x*y^2+x*z^2-x*z*t-y*t^2+y*z*t)+b*c*(x-t)*(x*z+y*t)+c*c*(z+t)*(y*t-x*z)+d*c*(y-z)*(x*z+y*t)+(e+f)*c*(x-y)*(y*t-x*z)

In [53]:
sing_cubics = (-1) * (-c + f) * (-c + e) * c * (c + f) * (c + e) * (-e + f)^2 * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f) * (b*c - c*f + e*f) * (b*c - c*e + e*f) * (b*c - c*d + 2*e*f) * (b*c - c^2 + e*f) * (b*c^2 + c^2*d + b*c*f - 2*c^2*f - c*d*f + 2*e*f^2) * (b*c^2 + c^2*d + b*c*e - 2*c^2*e - c*d*e + 2*e^2*f) * (-b*c^3 - 2*b*c^2*d + c^3*d + b*c^2*e + c^2*d*e + b*c^2*f + c^2*d*f + 3*b*c*e*f - 4*c^2*e*f - 3*c*d*e*f + 4*e^2*f^2)

In [54]:
line = Line([y, z])

In [55]:
eqn_SE1 = P(cubic_new.subs({b:-(c*c+e*f)/c }) )

In [56]:
sing_cubics_SE1 = P(sing_cubics.subs({b:-(c*c+e*f)/c }))

In [57]:
SE1 = Cubic(eqn_SE1, line, sing_cubics_SE1.factor())

In [58]:
SE1.eckardt_points

[(0, 0, 0, 1)]

In [59]:
%time adm_SE1 = SE1.find_admissible_projectivities()

CPU times: user 1.56 s, sys: 181 ms, total: 1.74 s
Wall time: 10.5 s


In [60]:
len(adm_SE1)

576

In [61]:
%time simm_SE1 = SE1.find_simmetries(adm_SE1)

CPU times: user 1.03 s, sys: 177 ms, total: 1.21 s
Wall time: 4.66 s


In [62]:
len(simm_SE1)

2

In [63]:
G_SE1 = Group(simm_SE1)

In [64]:
G_SE1.is_group()

True

In [65]:
G_SE1.get_order()

2

In [66]:
G_SE1.is_abelian()

True

In [67]:
G_SE1.get_divisors_of_order()

[1, 2]

In [68]:
G_SE1.get_order_of_elements()

[1, 2]

In [69]:
G_SE1.apply_to(SE1.eckardt_points[0])

[(0, 0, 0, 1), (0, 0, 0, 1)]

In [110]:
SE1.find_admissible_permutations()[1].index(['E1', 'G4', 'E3', 'G6', 'F24'])

95

In [315]:
def find_conditions_for_subfamilies(cubic, sing_cubics, projectivities, simmetries):
    mon = ((x+y+z+t)^3).monomials()
    conditions = []
    sing_cubics_factored = sing_cubics.factor()
    Eck = lcm([el.conditions for el in cubic.tritangent_planes if el.conditions != 0])
    for M in [proj for proj in projectivities if proj not in simmetries]:
        print(projectivities.index(M))
        sost = change_coord(M)
        new_cubic = remove_sing_factors(cubic.eqn.subs(sost), sing_cubics_factored)    
        minor = matrix([[new_cubic.coefficient(mn) for mn in mon], [cubic.eqn.coefficient(mn) for mn in mon]]).minors(2)
        minor = list(set([remove_sing_factors(el, sing_cubics_factored) for el in minor if el !=0]))
        ideale = cubic.P.ideal(minor)
        del_d_ideale = ideale.elimination_ideal([c])
        prim_deco = del_d_ideale.radical().primary_decomposition()
        for ideal in prim_deco:
            if not is_ideal_valid(ideal, sing_cubics_factored, Eck):
                continue
            prim_deco2 = (ideale + ideal).radical().primary_decomposition()
            for ideal2 in prim_deco2:
                if is_ideal_valid(ideal2, sing_cubics_factored, Eck):
                        gener = tuple([remove_sing_factors(gen, sing_cubics_factored) for gen in ideal2.gens() if gen !=0])
                        conditions.append(gener)                        
    return list(set(conditions))

def is_ideal_valid(ideal, sing_cubics_factored, Eck):
    gener = tuple([remove_sing_factors(gen, sing_cubics_factored) for gen in ideal.gens() if gen != 0])
    if 1 not in gener:
        for cond in gener:
            if cond.divides(Eck):
                return False
    return True

In [ ]:
conds = find_conditions_for_subfamilies(SE1, sing_cubics_SE1, adm_SE1[39:], simm_SE1)

In [279]:
M = adm_SE1[39]

In [291]:
sost = change_coord(M)

In [310]:
new_cubic = remove_sing_factors(SE1.eqn.subs(sost), sing_cubics_SE1.factor())

In [ ]:
mon = ((x+y+z+t)^3).monomials()

In [326]:
minor = matrix([[new_cubic.coefficient(mn) for mn in mon], [SE1.eqn.coefficient(mn) for mn in mon]]).minors(2)
minor = list(set([remove_sing_factors(el, sing_cubics_SE1.factor()) for el in minor if el !=0]))
ideale = P.ideal(minor)

In [ ]:
ideale = P.ideal(conds)

In [ ]:
ideale.elimination_ideal([c]).radical().primary_decomposition()

In [274]:
(ideale+_[6]).radical().primary_decomposition()

[Ideal (f, d - e, c + e) of Multivariate Polynomial Ring in x, y, z, t, b, c, d, e, f, l, m over Rational Field,
 Ideal (f, c) of Multivariate Polynomial Ring in x, y, z, t, b, c, d, e, f, l, m over Rational Field]

In [127]:
Q = PolynomialRing(QQ, [c,d,e,f])

In [132]:
idea = Q.ideal(conds[0])

In [ ]:
idea.triangular_decomposition()